In [1]:
import mlutils
import warnings
import os
import pandas as pd
warnings.filterwarnings(action="ignore")
os.getcwd()

station = pd.read_csv(
    "assets/seoul_bike_station_01_12.csv", encoding="CP949", index_col=0
)
near_bus = pd.read_csv(
    "assets/near_bus_500m.csv", encoding="CP949", index_col=0
)
seoul_bike = pd.read_parquet(
    "/Users/yangwoolee/git_repo/seoul_bike_dash/parquet/220607_bike_record.parquet.gzip"
)
sub_info = pd.read_csv(
    "assets/sub_and_bike_info.csv", encoding="CP949", index_col=0
)

search_info = pd.read_csv(
    "assets/search_info.csv", encoding="CP949", index_col=0
)
moon_light = mlutils.moon_light(station,near_bus,seoul_bike,sub_info)

In [ ]:
bike= search_info[search_info['value'] <3000]

for i in range(100) :
    dep = search_info.sample(1)
    dep_id = dep['value'].iloc[0]
    arr = bike.sample(1)
    arr_id = arr.index[0] 
    try : 
        moon_light.route_data(dep_id,arr_id)
    except :
        print('error :',dep, arr)
        break
print('문제 없음!!')


In [20]:

# id를 대여소 번호로 바꿔줌
def arrival_info( arr_id: int) -> pd.DataFrame:
    return station.iloc[[arr_id]]

a = mlutils.search_by_text(search_info,'복정역')
# moon_light.departure_info(4508)

# arrival_info(529)
print(a)

      value                 label
1737   1737           복정역 1번 출구 앞
2609   2609  복정역 2번출구 후문 (장지치안센터)
198    3198               8호선 복정역
588    3588               분당선 복정역
74     4074          N13 복정역환승주차장
75     4075      N13 복정역환승센터3번승강장
643    4643      N32 복정역환승센터3번승강장
720    4720          N32 복정역환승주차장
808    4808      N37 복정역환승센터3번승강장
918    4918          N37 복정역환승주차장


> ### 디버그 용도

In [2]:
import mlutils
import warnings
import os
import pandas as pd

warnings.filterwarnings(action="ignore")
os.getcwd()

station = pd.read_csv(
    "assets/seoul_bike_station_01_12.csv", encoding="CP949", index_col=0
)
near_bus = pd.read_csv("assets/near_bus_500m.csv", encoding="CP949", index_col=0)
seoul_bike = pd.read_parquet(
    "/Users/yangwoolee/git_repo/seoul_bike_dash/parquet/220607_bike_record.parquet.gzip"
)
sub_info = pd.read_csv("assets/sub_and_bike_info.csv", encoding="CP949", index_col=0)

search_info = pd.read_csv("assets/search_info.csv", encoding="CP949", index_col=0)
moon_light = mlutils.moon_light(station, near_bus, seoul_bike, sub_info)

#  def route_data(dep_id, arr_id):

dep_id = 4074
arr_id = 2609
dep_info = moon_light.departure_info(dep_id)  # dataframe
arr_info = moon_light.arrival_info(arr_id)  # dataframe


if dep_id < 3000:
    print("자전거")
    bike = moon_light.route_coor(dep_info, arr_info)
    record_info = moon_light.record_info(dep_info.index[0], arr_info.index[0], 50)
    # [{"bike": ["당산 육갑문", "목동1단지아파트 118동 앞"]}]
    route_info = [
        {
            "bike": [
                dep_info["st_name"].iloc[0],
                arr_info["st_name"].iloc[0],
                record_info[0],
                record_info[1],
            ]
        }
    ]

    retu = dict(bike=bike, route_info=route_info)

elif dep_id > 4000:
    print("버스")

    # * -- 출도착 버스 정보-- *
    dep_arr_bus_info = moon_light.finding_start_end(dep_info, arr_info)

    if dep_arr_bus_info.empty:
        print("해당 목적지로 이동가능한 야간버스가 없습니다.")
        retu= {"error": "해당 목적지로 이동가능한 야간버스가 없습니다."}

    # * -- 경로 선택 및 경로 노선 추출 -- *
    bus_whole, waypoint, bus_start_end = moon_light.bus_route_info(dep_arr_bus_info)

    if bus_whole.empty == False:
        # * -- 좌표 받기 -- *
        arr_trans = bus_start_end.iloc[[1]]

        arr_bike_info = moon_light.near_bus_bike_info(arr_trans, arr_info)

        if arr_bike_info.empty:
            retu= {"error": "이용가능한 따릉이 대여소가 없거나, 목적지와 도착지간 대여기록이 50건 미만입니다."}

        arr_bike_info = arr_bike_info.sort_values(by="num", ascending=False)

        bus = moon_light.bus_route_coor(bus_start_end, waypoint)
        center = moon_light.center_data(bus)
        walk = moon_light.route_coor(arr_trans, arr_bike_info.iloc[[0]])
        bike = moon_light.route_coor(arr_bike_info.iloc[[0]], arr_info)
        record_info = moon_light.record_info(
            arr_bike_info["st_id"].iloc[0], arr_info.index[0], 50
        )

        # # -- 자전거 길 네이버 지도로도 표현 -- #
        # a = arr_bike_info.iloc[[0]][["latitude", "longtitude"]]
        # b = arr_info.reset_index()[["latitude", "longtitude"]]
        # start_end = pd.concat([a, b], axis=0)
        # bus # bus_direct

        # if record_info == [0]:
        #     retu= {"error": "출발 대여소와 도착 대여소가 동일합니다.(2)"}

        route_info = [
            {
                "bus": [
                    bus_start_end["name"].iloc[0],
                    bus_start_end["name"].iloc[1],
                ],
                "bike": [
                    arr_bike_info["st_name"].iloc[0],
                    arr_info["st_name"].iloc[0],
                    record_info[0],
                    record_info[1],
                ],
            },
        ]

    #
    else:
        print("버스 경로에서 자전거 경로로 변경")
        arr_trans = bus_start_end.iloc[[0]]
        arr_bike_info = moon_light.near_bus_bike_info(arr_trans, arr_info)
        arr_bike_info = arr_bike_info.sort_values(by="num", ascending=False)
        dep_bike_info = moon_light.near_bus_bike_info(bus_start_end.iloc[[0]], arr_info)
        bike2 = moon_light.route_coor(dep_bike_info, arr_info)
        center2 = moon_light.center_data(bike2)
        record_info2 = moon_light.record_info(
            dep_bike_info["st_id"].iloc[0], arr_info.index[0], 50
        )
        bike2_info = (
            {
                "bike2": [
                    dep_bike_info["st_name"].iloc[0],
                    arr_info["st_name"].iloc[0],
                    record_info2[0],
                    record_info2[1],
                ]
            },
        )
    retu= dict(
        bike=bike2,
        center=center2,
        route_info=bike2_info,
    )
    print(retu)
    #

    dep_bike_info = moon_light.near_bus_bike_info(bus_start_end.iloc[[0]], arr_info)

    if dep_bike_info.empty:
        retu= {"error": "이용가능한 따릉이 대여소가 없거나, 목적지와 도착지간 대여기록이 50건 미만입니다."}

    dep_bike_info = dep_bike_info.sort_values(by="num", ascending=False)
    if dep_bike_info.empty == False:
        bike2 = moon_light.route_coor(dep_bike_info, arr_info)
        center2 = moon_light.center_data(bike2)
        record_info2 = moon_light.record_info(
            dep_bike_info["st_id"].iloc[0], arr_info.index[0], 50
        )
        bike2_info = (
            {
                "bike2": [
                    dep_bike_info["st_name"].iloc[0],
                    arr_info["st_name"].iloc[0],
                    record_info2[0],
                    record_info2[1],
                ]
            },
        )
        route_info.extend(bike2_info)

    else:
        bike2 = [""]
        center2 = [""]

    retu= dict(
        bus=bus,
        walk=walk,
        bike=bike,
        center=center,
        bike2=bike2,
        center2=center2,
        route_info=route_info,
    )


else:
    print("지하철")
    # * -- 자전거 대여소 정보 추출 -- *
    near_bus_bike_info = moon_light.near_bus_bike_info(dep_info, arr_info).sort_values(
        by="dist"
    )
    if near_bus_bike_info.empty :
        print('near_bus_bike_info is empty')
    departure_bike_station = pd.DataFrame(near_bus_bike_info.iloc[0]).T

    # * -- 자전거 경로(사실 보행로 추천이라는 사실~)-- *
    sub = moon_light.route_coor(dep_info, departure_bike_station)
    bike = moon_light.route_coor(departure_bike_station, arr_info)
    record_info = moon_light.record_info(
        departure_bike_station["st_id"].iloc[0], arr_info.index[0], 50
    )
    # sub
    # [{"sub": ["당산역"], "bike": ["당산 육갑문", "목동1단지아파트 118동 앞"]}]
    route_info = [
        {
            "sub": [dep_info["sub_name"].iloc[0]],
            "bike": [
                departure_bike_station["st_name"].iloc[0],
                arr_info["st_name"].iloc[0],
                record_info[0],
                record_info[1],
            ],
        }
    ]
    retu = dict(sub=sub, bike=bike, route_info=route_info)


버스
도착대여소와 정류장이 직선거리 1.5km 이내에 있습니다.
        bike_id  bus  order     id      name  longtitude   latitude
74  [2605,2607]  N13     74  24658  복정역환승주차장  127.128144  37.470728
버스를 이용하지 않아도 되는 동선입니다.
버스 경로에서 자전거 경로로 변경
{'bike': array([[ 37.47127953, 127.12707408],
       [ 37.47122676, 127.12714908],
       [ 37.47122676, 127.12714908],
       [ 37.47109622, 127.12702965],
       [ 37.47109622, 127.12702965],
       [ 37.47099623, 127.12702965],
       [ 37.47099623, 127.12702965],
       [ 37.47055739, 127.12703522],
       [ 37.47055739, 127.12703522],
       [ 37.47061572, 127.12712965]]), 'center': array([ 37.47099623, 127.12702965]), 'route_info': ({'bike2': ['복정역 1번 출구 앞', '복정역 2번출구 후문 (장지치안센터)', 25.0, 250]},)}
출발 대여소와 도착 대여소가 동일합니다.


NameError: name 'route_info' is not defined

In [40]:
retu

{'bus': [''],
 'walk': [''],
 'bike': array([[ 37.47061572, 127.12712965],
        [ 37.47055739, 127.12703522],
        [ 37.47055739, 127.12703522],
        [ 37.47099623, 127.12702965],
        [ 37.47099623, 127.12702965],
        [ 37.47109622, 127.12702965],
        [ 37.47109622, 127.12702965],
        [ 37.47122676, 127.12714908],
        [ 37.47122676, 127.12714908],
        [ 37.47127953, 127.12707408]]),
 'center': array([ 37.47109622, 127.12702965]),
 'bike2': array([[ 37.47061572, 127.12712965],
        [ 37.47061572, 127.12712965]]),
 'center2': array([ 37.47061572, 127.12712965]),
 'route_info': [{'bike': ['복정역환승주차장', '복정역 1번 출구 앞', 0, 0]},
  {'bike2': ['복정역 2번출구 후문 (장지치안센터)', '복정역 1번 출구 앞', 25.0, 250]},
  {'bike2': ['복정역 2번출구 후문 (장지치안센터)', '복정역 1번 출구 앞', 25.0, 250]},
  {'bike2': ['복정역 2번출구 후문 (장지치안센터)', '복정역 2번출구 후문 (장지치안센터)', 0, 1]},
  {'bike2': ['복정역 2번출구 후문 (장지치안센터)', '복정역 2번출구 후문 (장지치안센터)', 0, 1]},
  {'bike2': ['복정역 2번출구 후문 (장지치안센터)', '복정역 2번출구 후문 (장지치안센터)', 0, 1]},

In [19]:
import numpy as np
def haversine_np(
        lon1: float, lat1: float, lon2: float, lat2: float
    ) -> np.array:
        """
        Calculate the great circle distance between two points
        on the earth (specified in decimal degrees)

        All args must be of equal length.

        """
        lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = (
            np.sin(dlat / 2.0) ** 2
            + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
        )

        c = 2 * np.arcsin(np.sqrt(a))
        m = 6367 * c * 1000
        return m

def finding_start_end(
     departure_info: pd.DataFrame, arrival_info: pd.DataFrame
) -> pd.DataFrame:
    """
    최종적으로 자전거 대여소끼리 이동하는 경로를 찾는 함수
    목적지 대여소와 가까운 버스정류소를 찾고 그 근처의 대여소를 추천해준다.

    bus에서 내린 뒤 대여소로 이동하는 경우를 산정함

    departure_info : 출발지 검색용 변수
    ending_bike : 목적지 대여소 번호
    arrival_info = 목적지 대여소 정보
    """

    bus_num = departure_info["bus"].iloc[0]
    bus_name = departure_info["name"].iloc[0]
    departure_info = near_bus.query("bus==@bus_num & name==@bus_name")
    # 목적지 대여소와 직선거리 1.5km 이내 버스대여소 모두 선택
    # 결과 : bus_route_near_bike
    result_km = haversine_np(
        arrival_info["longtitude"].values,
        arrival_info["latitude"].values,
        near_bus["longtitude"].values,
        near_bus["latitude"].values,
    )
    near_bus_for_dist = near_bus.copy()
    near_bus_for_dist["dist"] = result_km
    bus_route_near_bike = (
        near_bus_for_dist[near_bus_for_dist["dist"] < 1500]
        .dropna()
        .sort_values(by="id")
    )
    # departure_info와 arrival_info 인근 버스 정류장과 겹치는 버스 번호 찾기
    # ex) 합정역 => 754 자전거 대여소 인 경우 754 근처를 지나가는 버스 N26과 N62를 결과로 제공함.
    bus_route_inter = np.intersect1d(
        bus_route_near_bike["bus"].unique(), departure_info["bus"]
    ).tolist()

    # 주변에 겹치는 결과가 없으면 검색 종료
    if bus_route_inter == []:
        print("목적지와 연결된 검색노선 없음.")

        return pd.DataFrame([])

    # 겹치는 노선에 대한 정보만 불러온더
    # 합정의 경우 N26,N62외에도 다양한 야간 버스가 지나친다. 그중 N26,N62버스 만 불러온다.
    # starting_bus = departure_info.query("bus == @bus_route_inter")

    starting_bus = departure_info
    ending_bus = bus_route_near_bike.query("bus == @bus_route_inter")

    # 정류장 시작,종료 지점 모으기.. 머리 간질간질해 죽는줄..
    # A에서 B갈때 거치는 정류장과 B에서 A갈때 거치는 정류장이 다를 수 있음.
    # 정확한 버스노선을 찾기 위해서는 버스 방향이 상행인지 하행인지가 중요하다.
    # 상행과 하행을 찾는 방법에 대한 방법이다.

    bus_df = pd.DataFrame(columns=ending_bus.columns)


    # bus_route_inter : A->B갈때 여러 N버스를 타고 갈 수 있음
    # 합정 => 754인 경우 N26과 N62 둘 다 가능
    for num in bus_route_inter:
        starting_order = starting_bus.query("bus==@num")
        ending_order = ending_bus.sort_values(by=["dist", "order"]).query(
            "bus==@num"
        )
        print(ending_order)
        print(starting_order)

        # ending_order = ending_bus.sort_values(by=["dist", "order"]).query(
        #     "bus==@num"
        # )[:2]

        # 상행(A->B)인 경우 B가 A보다 bus 정류장 번호(column : bus_order)가 더 크다.
        # 하행(B->A)인 경우 A가 B보다 ""

        # 그 이유는 버스가 이동할때마다 +1씩 증가하기 때문임. 운행중에는 번호가 계속 증가하는 순으로 갔다가
        # 차고지에서 다시 0으로 리셋하고 출발한다는 생각을 하면 쉬움.

        # 따라서 하행인 경우 ending_order .max - starting.max > 0 이며 상행인 경우 그 반대임.

        # 하행노선 찾기
        if ending_order["order"].max() - starting_order["order"].max() >= 0:
            df = starting_order.sort_values(by="order", ascending=False)[:1]
            bus_df = pd.concat([bus_df, df], axis=0)
            # bus_df = bus_df.append(df) ## concat 에러뜨면 append로 다시 바꾸기
            df = ending_order.sort_values(by="order", ascending=False)[:1]
            bus_df = pd.concat([bus_df, df], axis=0)
            # print(f"{num}하행 노선입니다.")

        # 상행노선 찾기
        elif ending_order["order"].max() - starting_order["order"].max() < 0:
            df = starting_order.sort_values(by="order")[:1]
            bus_df = pd.concat([bus_df, df], axis=0)
            df = ending_order.sort_values(by="order")[:1]
            bus_df = pd.concat([bus_df, df], axis=0)
            # print("상행 노선입니다.")

    # 출발지 위치가 목적지 1.5km이내에 있는 경우 버스 경로검색 안하도록 조치
    if starting_order["id"].iloc[0] in bus_route_near_bike["id"].tolist():
        bus_df = starting_order.sort_values(by="order", ascending=False)[:1]
        print("도착대여소와 정류장이 직선거리 1.5km 이내에 있습니다.")
        print(bus_df)
        # 아래 함수랑 연동시켜보자
        return bus_df

    # 아래 함수랑 연동시켜보자
    return bus_df  #  노선별 출발 => 도착를 return 한다.

finding_start_end(dep_info, arr_info)

                                bike_id  bus  order     id             name  \
441                          [227,4594]  N26     22  19218         양평동한솔아파트   
440                           [744,766]  N26     21  19217            안양천입구   
526                       [227,744,766]  N26    111  19215       영등포아리수정수센터   
527       [700,773,1169,1171,2703,3761]  N26    112  16001       염창역.서울도시가스   
439  [700,773,1169,1171,2703,3761,5067]  N26     20  16002       염창역.서울도시가스   
442                          [228,4594]  N26     23  19214  양평한신아파트.선유도공원입구   
525                          [228,4594]  N26    110  19216  양평한신아파트.선유도공원입구   

     longtitude   latitude         dist  
441  126.890347  37.543917  1035.418263  
440  126.883667  37.547306  1067.502726  
526  126.884113  37.547524  1099.156427  
527  126.875881  37.547097  1135.362300  
439  126.875005  37.546964  1157.626816  
442  126.894428  37.541649  1221.687569  
525  126.894879  37.541864  1267.248692  
                        bike_id

,bike_id,bus,order,id,name,longtitude,latitude,dist
524,"[247,267,268,272]",N26,109,19176,당산역,126.90241,37.534863,NaN
527,"[700,773,1169,1171,2703,3761]",N26,112,16001,염창역.서울도시가스,126.875881,37.547097,1135.3623


In [ ]:

def bus_route_info( bus_route: pd.DataFrame) -> pd.DataFrame:
    if bus_route.iloc[-1]["id"] == bus_route.iloc[0]["id"]:
        print("버스를 이용하지 않아도 되는 동선입니다.")
        return (
            pd.Series([]),
            pd.Series([]),
            bus_route,
        )  ## 세번째에 bus_route를 넣은 이유는 bike2를 이용하기 위함임.
    else:
        # 범위선택
        bus_order_start = bus_route["order"].iloc[0]
        bus_order_end = bus_route["order"].iloc[-1]
        bus_name = bus_route["bus"].iloc[0]
        print(bus_order_start)
        print(bus_order_end)
        print(bus_name)

        # if route1 < route2:
        #     bus_order_start = route1
        #     bus_order_end = route2
        # else:
        #     bus_order_start = route2
        #     bus_order_end = route1
        # 자료 sorting
        route_whole = near_bus.query(
            "@bus_order_start <= order <= @bus_order_end & bus == @bus_name"
        )[["name", "longtitude", "latitude", "order", "bike_id"]].sort_values(
            by="order"
        )
        print(route_whole)

        route_start = route_whole.iloc[0]
        route_end = route_whole.iloc[-1]
        route_start_and_end = pd.concat([route_start, route_end], axis=1).T

        waypoint = route_whole.iloc[int(len(route_whole) / 2)]

    return route_whole, waypoint, route_start_and_end

bus_route_info(dep_arr_bus_info)

In [ ]:

    # * -- 경로 선택 및 경로 노선 추출 -- *
    bus_whole, waypoint, bus_start_end = moon_light.bus_route_info(dep_arr_bus_info)

    # * -- 좌표 받기 -- *


    arr_trans = bus_start_end.iloc[[1]]

    dep_bike_info = moon_light.near_bus_bike_info(
    bus_start_end.iloc[[0]], arr_info
    ).sort_values(by="num", ascending=False)

    arr_bike_info = moon_light.near_bus_bike_info(
    arr_trans, arr_info
    ).sort_values(by="num", ascending=False)

    dep_arr_bike_coor = moon_light.route_coor(dep_bike_info, arr_info)


    bus_coor = moon_light.bus_route_coor(bus_start_end, waypoint)
    walk_coor = moon_light.route_coor(
        arr_trans, arr_bike_info.iloc[[0]]
    )
    bike_coor = moon_light.route_coor(arr_bike_info.iloc[[0]], arr_info)
    retu= dict(
        dep_arr_bike_coor=dep_arr_bike_coor,
        bus_coor=bus_coor,
        walk_coor=walk_coor,
        bike_coor=bike_coor,
    )
else:
    print("지하철")
    # * -- 자전거 대여소 정보 추출 -- *
    near_bus_bike_info = moon_light.near_bus_bike_info(
        dep_info, arr_info
    ).sort_values(by="dist")
    departure_bike_station = pd.DataFrame(near_bus_bike_info.iloc[0]).T

    # * -- 자전거 경로(사실 보행로 추천이라는 사실~)-- *
    walk_coor = moon_light.route_coor(dep_info, departure_bike_station)
    bike_coor = moon_light.route_coor(departure_bike_station, arr_info)
    retu= dict(
        walk_coor=walk_coor, bike_coor=bike_coor
    )

> ### Card에 넣을 내용

In [4]:

from typing import Dict

def record_check(dep_id: int,arr_id: int,count : int = 50) -> pd.DataFrame :
    BM = (seoul_bike['st_id1'] == dep_id) & (seoul_bike['st_id2'] == arr_id)
    raw_data_1 = seoul_bike[BM]
    BM = (seoul_bike['st_id1'] == arr_id) & (seoul_bike['st_id2'] == dep_id)
    raw_data_2 = seoul_bike[BM]
    concat_data = pd.concat([raw_data_1,raw_data_2],axis=0)

    if len(concat_data) <= count :
        return pd.DataFrame([])

    
    all_time = concat_data['riding_time'].value_counts().sort_values(ascending=False) # 도달한 시간대
    total_borrow =len(concat_data)

    k = []
    i = 2
    # 일정 비율 이상인 시간대만 구하기
    while len(k) < 1:
        k = all_time[all_time >= (total_borrow / i)]
        i = i * 1.5
        
    ### 대여시간대
    time_line = k.index

    ### 대여기록
    counts = k.values

    total_counts = k.sum()

    ### 대여시간 * 대여기록
    total_time = sum([a * b for a, b in zip(time_line, counts)])

    # 평균 시간
    avg_time = total_time / total_counts

    # 올림
    result = round(avg_time, 0)

    return [result,total_borrow]
dep_info = moon_light.departure_info(4443)
arr_info = moon_light.arrival_info(528)


record_check(arr_bike_info['st_id'].iloc[0],arr_info.index[0])

NameError: name 'arr_bike_info' is not defined

In [28]:
# bus # bus_direct
[{"bus":[bus_start_end['name'].iloc[0],bus_start_end['name'].iloc[1]],'bike':[arr_bike_info['st_name'].iloc[0], arr_info['st_name'].iloc[0]]}, {'bike2' : [dep_bike_info['st_name'].iloc[0], arr_info['st_name'].iloc[0]]}]

# sub
[{"sub":['당산역'],'bike':['당산 육갑문', '목동1단지아파트 118동 앞']}]
[{"sub":[dep_info['sub_name'].iloc[0]],'bike':[departure_bike_station['st_name'].iloc[0], arr_info['st_name'].iloc[0]]}]


# bike
[{'bike':['당산 육갑문', '목동1단지아파트 118동 앞']}]
[{'bike' : [dep_info['name'].iloc[0],arr_info['st_name'].iloc[0]]}]

33.0

In [8]:
print("bus_start_end", bus_start_end['name'].iloc[0])
print('')
print("arr_trans", arr_trans.values)
print('')
print("arr_bike_info", arr_bike_info['st_name'])
print('')
print("arr_info", arr_info['st_name'].iloc[0])
print('')
print("dep_info", dep_info)
print('')
print("dep_bike_info", dep_bike_info)
print('')
print("departure_bike_station", departure_bike_station)
print('')

bus_start_end 당산역

arr_trans [['염창역.서울도시가스' 126.875881 37.547097 112 '[700,773,1169,1171,2703,3761]']]

arr_bike_info 1    염창역 3, 4번 출구 사이
2          염창역 1번 출구
0    KB국민은행 염창역 지점 앞
Name: st_name, dtype: object

arr_info 목동1단지아파트 118동 앞 (월촌초등학교 정류소 옆

dep_info                         bike_id  bus  order     id name  longtitude   latitude
443  [247,267,268,272,290,3201]  N26     24  19172  당산역  126.900719  37.534589

dep_bike_info   st_name   latitude  longtitude  st_id  time  num    dist
0   당산육갑문  37.535339  126.903679    272  33.0  109  3.03km



NameError: name 'departure_bike_station' is not defined

> ### 이름으로 버스 정류장과 대여소 선택